# NDVI Calculation

In [3]:
import fiona
import sentinelsat
import ipywidgets as widgets
import numpy as np
import os
import lithops
import random
import rasterio
import re
import tempfile
import zipfile
import subprocess
import glob
import json
from rio_cogeo import cogeo

from collections import defaultdict
from datetime import date
from fiona.io import ZipMemoryFile
from matplotlib import pyplot as plt
from rasterio.io import MemoryFile
from zipfile import ZipFile
from ipyleaflet import Map, basemaps, basemap_to_tiles
from lithops.storage import Storage

from utils import notebook as notebook_utils
from io_utils.ndvi import get_ndvi_params, ndvi_calculation, ndvi_tile_sentinel, get_subset_raster, lonlat_to_utm, get_poly_within
from io_utils.plot import tiff_overview, plot_map

os.environ['CURL_CA_BUNDLE'] = '/etc/ssl/certs/ca-certificates.crt'

Set the environmental variables *SENTINEL_USERNAME* and *SENTINEL_PASSWORD* to match your Sentinel-2 credentials. You can register and access data for free at https://sentinel.esa.int/web/sentinel/sentinel-data-access/registration:

In [4]:
SENTINEL_USERNAME = ...
SENTINEL_PASSWORD = ...
STORAGE_BUCKET = ...

In [5]:
%matplotlib inline

In [6]:
cloud_storage = Storage()

## Input parameters

Select the date interval in which tiles will be processed:

In [7]:
from_day, to_day = notebook_utils.pick_date_range()

DatePicker(value=datetime.date(2019, 11, 1), description='From day')

DatePicker(value=datetime.date(2019, 11, 30), description='To day')

Select the tile's cloud percentage threshold:

In [8]:
percentage = notebook_utils.pick_percentage_slider()

IntSlider(value=15, continuous_update=False, description='Porcentaje de nubosidad')

Select the area which delimites the tiles you want to process (left click to mark a point in the map, right click to erase current selection):

In [9]:
map_region = notebook_utils.MapRegion()

Map(basemap={'url': 'http://{s}.tile.openstreetmap.fr/hot/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': '&c…

## Get Sentinel-2 metadata

In [ ]:
#locations = map_region.get_region()
# TODO hardcoded for dev purposes
locations = [[-1.32110595703125, 37.57329031970199],
 [-2.0681762695312504, 37.684227882053044],
 [-1.636962890625, 38.24289903439589],
 [-0.7745361328125, 38.12199840979802],
 [-1.32110595703125, 37.57329031970199]]
locations


Using the selected parameters, get the identifiers of the selected tiles from Sentinel-2:

In [ ]:
geo_json_area = {"features": [{"geometry": {"coordinates": [locations], "type": "Polygon"}, "properties": {}, "type": "Feature"}], "type": "FeatureCollection"}

In [ ]:
sentinel_api = sentinelsat.SentinelAPI(user=SENTINEL_USERNAME,
                                       password=SENTINEL_PASSWORD)
footprint = sentinelsat.geojson_to_wkt(geo_json_area)
products = sentinel_api.query(footprint,
                              date=(from_day.value, to_day.value),
                              platformname='Sentinel-2',
                              producttype=('S2MS2Ap', 'S2MSI1C'),
                              cloudcoverpercentage=(0, percentage.value))
geojson_products = sentinel_api.to_geojson(products)

In [ ]:
# For debug purposes, we will compute one tile. Skip this cell to compute all tiles.
geojson_products = [geojson_products[0]]

In [ ]:
print('Number of tiles: {}'.format(len(geojson_products)))

## Athmospheric correction using Serverful Lithops

Here we will download tile images from Sentinel2 using the previously selected configuration and apply athmospheric correction.

This process is not parallelizable and lasts for over 20 minutes, so it is not suited for serverless functions. We will use Lithops Standalone instead, which uses serverful instances that haven't time limits.

The runtime used packs Sentinel2 software and IBM Cloud Functions Python3.7 handler in a Dockerfile located in `sentinel2_runtime/Dockerfile`.

In [ ]:
def jp2_to_cog(band_src_path):
    """
    Transform a sentinel2 band (.jp2) to GeoTiff (.tif)
    """
    config = dict(NUM_THREADS=100, GDAL_TIFF_OVR_BLOCKSIZE=128)

    output_profile = {
        "driver": "GTiff",
        "interleave": "pixel",
        "tiled": True,
        "blockxsize": 256,
        "blockysize": 256,
        "compress": "DEFLATE",
    }

    cog_path = f"{band_src_path[band_src_path.rfind('/')+1:band_src_path.rfind('.')]}.tif"
    cogeo.cog_translate(
        band_src_path,
        cog_path,
        output_profile,
        nodata=0,
        in_memory=False,
        config=config,
        quiet=True,
    )

    return cog_path

In [ ]:
def perform_atmospheric_correction(product_geojson, storage):
    product = product_geojson['properties']
    tile = product['filename'][39:44]
    date = product['filename'][11:19]

    # Check if tile is already processed stored in COS, return if it is
    #keys = storage.list_keys(bucket=STORAGE_BUCKET)
    keys = []
    pattern = r".*" + re.escape(date) + r".*" + re.escape(tile) + r".*\.geojson"
    filtered = [file for file in keys if re.search(pattern, file)]
    if filtered:
        print('Tile already in COS: {}'.format(filtered))
        remote_geotiff = filtered.pop()
        return remote_geotiff

    # Download Sentinel-2 tile
    sentinel_api = sentinelsat.SentinelAPI(user=os.environ["SENTINEL_USERNAME"],
                                           password=os.environ["SENTINEL_PASSWORD"])

    sentinel_product_dir = product['filename']
    tmpdir = tempfile.gettempdir()
    d_meta = sentinel_api.download(product['uuid'], directory_path=tmpdir)
    
    # Extract and remove zip file
    zip_ref = zipfile.ZipFile(d_meta['path'])
    zip_ref.extractall(tmpdir)
    zip_ref.close()
    #os.remove(d_meta['path'])

    # Atmospheric correction
    sentinel_product_dir = os.path.join(tmpdir, product['filename'])
    corrected_images = glob.glob(f"*2A_{date}*_T{tile}_*.SAFE/GRANULE/*/IMG_DATA/R10m/*B0[48]*.jp2")
    atmospheric_corrected = corrected_images[0] if len(corrected_images) > 0 else None

    if not atmospheric_corrected:
        print(f'Doing the atmospheric correction for {sentinel_product_dir}')
        try:
            cmd = ['L2A_Process --resolution 10 {}'.format(sentinel_product_dir)]
            val = subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True, universal_newlines=True)
            corrected_images = glob.glob(f"*2A_{date}*_T{tile}_*.SAFE/GRANULE/*/IMG_DATA/R10m/*B0[48]*.jp2")
            print(f'Atmospheric correction finished {val}')
        except subprocess.CalledProcessError as e:
            print(e.returncode)
            print(e.output)
            print(e.stderr)
            raise(e)


    # Translate bands in .jp2 to GeoTiff format
    band_files = []
    band4 = glob.glob(os.path.join(tmpdir, '*L2A_{}*_T{}*.SAFE/GRANULE/*/IMG_DATA/R10m/*B04*'.format(date, tile))).pop()
    band8 = glob.glob(os.path.join(tmpdir, '*L2A_{}*_T{}*.SAFE/GRANULE/*/IMG_DATA/R10m/*B08*'.format(date, tile))).pop()

    if band4 is not None and band8 is not None:
        band4_tiff_file = f"{band4[band4.rfind('/')+1:band4.rfind('.')]}.tif"
        band8_tiff_file = f"{band8[band8.rfind('/') + 1:band8.rfind('.')]}.tif"
        jp2_to_cog(band4)
        jp2_to_cog(band8)
        band_files.append(band4_tiff_file)
        band_files.append(band8_tiff_file)
    
    print(band_files)

    # Merge both bands into a single geotiff
    combined_geotiff_key = band_files[0][0:22] + '_COMBINED.tif'
    with rasterio.open(band_files[0]) as src:
        profile = src.profile
        profile.update(count=len(band_files))

    with rasterio.open(combined_geotiff_key, 'w', **profile) as dst:
        for i, band_file in enumerate(band_files):
            with rasterio.open(band_file) as src:
                dst.write(src.read(1), i + 1)

    # Upload generated files to Cloud Storage
    with open(combined_geotiff_key, 'rb') as combined_geotiff_f:
        storage.put_object(bucket=STORAGE_BUCKET, key=combined_geotiff_key, body=combined_geotiff_f)
    product_meta_key = combined_geotiff_key + '.meta.json'
    storage.put_object(bucket=STORAGE_BUCKET, key=product_meta_key, body=json.dumps(product))

    return combined_geotiff_key

In [ ]:
lithops_standalone = lithops.StandaloneExecutor(runtime='aitorarjona/lithops_ndvi_sen2cor:1.0', workers=1, log_level='DEBUG')
extra_env = {'SENTINEL_USERNAME': SENTINEL_USERNAME,
             'SENTINEL_PASSWORD': SENTINEL_PASSWORD}
lithops_standalone.map(perform_atmospheric_correction, geojson_products, extra_env=extra_env)

In [ ]:
combined_keys = lithops_standalone.get_result()
print(combined_keys)

## NDVI Computation using Serverless Lithops

Now we will calculate NDVI index of tiles tha thave been downloaded and pre-processed before.

This process can be executed in parallel (for every tile) and in serverless functions.

In [ ]:
lithops_serverless = lithops.ServerlessExecutor(runtime='cloudbuttonans/geospatial-runtime:3.7-v4', runtime_memory=2048, log_level='DEBUG')

In [ ]:
# DEBUG
combined_keys = ['T30SWG_20191127T105309_COMBINED.tif']

In [ ]:
def ndvi(combined_key, storage):
    tmpdir = tempfile.gettempdir()
    dat = storage.get_object(bucket=STORAGE_BUCKET, key=combined_key, stream=True)
    out = os.path.join(tmpdir, 'out.tif')

    with rasterio.open(dat) as src:
        profile = src.profile
        profile.update(dtype='float32')
        profile.update(count=1)
        with rasterio.open(out, 'w', **profile) as dst:
            for _, window in src.block_windows(1):
                red = src.read(1, window=window).astype('float32')
                nir = src.read(2, window=window).astype('float32')
                ndvi = (np.where((nir + red) == 0., 0,
                                 (nir - red) / (nir + red))).astype('float32')
                dst.write(ndvi, 1, window=window)

    prefix = combined_key.rsplit('_', 1)[0]
    output_key = prefix + '_NDVI.tif'
    with open(out, 'rb') as output_f:
        storage.put_object(bucket=STORAGE_BUCKET, key=output_key, body=output_f)

    return output_key

In [ ]:
lithops_serverless.map(ndvi, combined_keys)

In [ ]:
ndvi_keys = lithops_serverless.get_result()
ndvi_keys = ['T30SWG_20191127T105309_NDVI.tif']  # DEBUG
ndvi_keys

In [ ]:
tile_select = notebook_utils.pick_tile(ndvi_keys)

In [ ]:
obj = cloud_storage.get_object(bucket=STORAGE_BUCKET, key='T30SWG_20191127T105309_NDVI.tif', stream=True)
fig, axs = plt.subplots(figsize=(20,15))
with rasterio.open(obj) as src:
    ij, window = random.choice(list(src.block_windows()))
    arr = src.read(1, window=window)
    plt.imshow(arr)